In [10]:
import numpy as np
import pandas as pd
import requests

from PB_funciones import nuevo_libro

In [19]:
# ${{secrets.APIKEY_ISBN}} #----------> No funcionan los secretos!!!

In [18]:
# Se crea el diccionario que contendrá los valores de los libros solicitados.
dict_books = {}


In [9]:
# Web scraping


# url de la página web a «escrapear»
url = 'https://es.wikipedia.org/wiki/Anexo:Libros_m%C3%A1s_vendidos'

# pasar "User-agent" para simular interacción con la página usando Navegador web
headers = {"User-agent": 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/79.0.3945.130 Safari/537.36'}

respuesta = req.get(url, headers=headers)

# El código de respuesta <200> indicará que todo salió bien
print(respuesta)

<Response [200]>
